# Sample Notebook 2 for Picasso
This notebook shows some basic interaction with the picasso library. It assumes to have a working picasso installation. To install jupyter notebooks in a conda picasso environment use `conda install nb_conda`.
The sample data was created using Picasso:Simulate. You can download the files here: http://picasso.jungmannlab.org/testdata.zip

## Load Localizations

In [1]:
from picasso import io
path = 'testdata_locs.hdf5'
locs, info = io.load_locs(path)

print('Loaded {} locs.'.format(len(locs)))

Loaded 11975 locs.


## Info file
The info file is now a list of dictionaries. Each step in picasso adds an element to the list.

In [2]:
for i in range(len(info)):
    print(info[i]['Generated by'])
    
# extract width and height:
width, height = info[0]['Width'], info[0]['Height']
print('Image height: {}, width: {}'.format(width, height))

Picasso simulate
Picasso Localize
Image height: 32, width: 32


## Filter localizations

Filter localizations, i.e., via sx and sy: Remove all localizations that are not within a circle around a center position.

In [3]:
sx_center = 0.82
sy_center = 0.82

radius = 0.04 

to_keep = (locs.sx-sx_center)**2 + (locs.sy-sy_center)**2 < radius**2

filtered_locs = locs[to_keep]
print('Length of locs before filtering {}, after filtering {}.'.format(len(locs),len(filtered_locs)))

Length of locs before filtering 11975, after filtering 9864.


## Saving localizations
Add new info to the yaml file and save everything.

In [4]:
import os.path as _ospath
# Create a new dictionary for the new info
new_info = {}
new_info["Generated by"] = "Picasso Jupyter Notebook"
new_info["Filtered"] = 'Circle'
new_info["sx_center"] = sx_center
new_info["sy_center"] = sy_center
new_info["radius"] = radius

info.append(new_info)

base, ext = _ospath.splitext(path)

new_path = base+'_jupyter'+ext


io.save_locs(new_path, filtered_locs, info)

print('{} locs saved to {}.'.format(len(filtered_locs), new_path))

9864 locs saved to testdata_locs_jupyter.hdf5.


## Manually export images
Use the picasso functions to render images.

In [5]:
# Get minimum / maximum localizations to define the ROI to be rendered 
import numpy as np
from picasso import render
import matplotlib.pyplot as plt

x_min = np.min(locs.x)    
x_max = np.max(locs.x)
y_min = np.min(locs.y)
y_max = np.max(locs.y)

viewport =  (y_min, x_min), (y_max, x_max)
oversampling = 10
len_x, image = render.render(locs, viewport = viewport, oversampling=oversampling, blur_method='smooth')
plt.imsave('test.png', image, cmap='hot', vmax=10)

# Cutom ROI with higher oversampling
viewport =  (5, 5), (10, 10)
oversampling = 20
len_x, image = render.render(locs, viewport = viewport, oversampling=oversampling, blur_method='smooth')
plt.imsave('test_zoom.png', image, cmap='hot', vmax=10)

## Calculate kinetics
Use the picasso functions to calculate kinetics.

In [7]:
from picasso import postprocess

# Note: to calculate dark times you need picked localizations of single binding sites
path = 'testdata_locs_picked_single.hdf5'
picked_locs, info = io.load_locs(path)

# Link localizations and calcualte dark times
linked_locs = postprocess.link(picked_locs, info, r_max=0.05, max_dark_time=1)
linked_locs_dark = postprocess.compute_dark_times(linked_locs)

print('Average bright time {:.2f} frames'.format(np.mean(linked_locs_dark.n)))
print('Average dark time {:.2f} frames'.format(np.mean(linked_locs_dark.dark)))

# Compare with simulation settings:
integration_time = info[0]['Camera.Integration Time']
tau_b = info[0]['PAINT.taub']
k_on = info[0]['PAINT.k_on']
imager = info[0]['PAINT.imager']
tau_d = 1/(k_on*imager)*10**9*1000

print('------')

print('ON Measured {:.2f} ms \t Simulated {:.2f} ms'.format(np.mean(linked_locs_dark.n)*integration_time, tau_b))
print('OFF Measured {:.2f} ms \t Simulated {:.2f} ms'.format(np.mean(linked_locs_dark.dark)*integration_time, tau_d))


Average bright time 2.10 frames
Average dark time 458.96 frames
------
ON Measured 630.49 ms 	 Simulated 500.00 ms
OFF Measured 137688.98 ms 	 Simulated 125000.00 ms
